In [1]:
try:
    from iapws import IAPWS97
except ImportError:
    !pip install iapws
    from iapws import IAPWS97

  Using cached iapws-1.5.4-py3-none-any.whl


In [2]:
try:
    from pyfluids import HumidAir, InputHumidAir
except ImportError:
    !pip install pyfluids
    from pyfluids import HumidAir, InputHumidAir

  Using cached pyfluids-2.7.1-py3-none-any.whl (23 kB)
  Using cached CoolProp-6.6.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.3 MB)


Air enters a 30-cm-diameter cooling section at 1 atm, 35°C, and 45 percent relative humidity at 18 m/s. Heat is removed from the air at a rate of 750 kJ/min. 

Determine: 
- (a) the exit temperature,
- (b) the exit relative humidity of the air, and
- (c) the exit velocity. 

Answers: 
- (a) 26.5°C,
- (b) 73.1 percent,
- (c) 17.5 m/s

In [18]:
D0_cm = 30
D0_m = D0_cm/100

p1_Pa = 101325
T1_C = 35 
phi_1_percent = 45
vel1_mPerS = 18

Qout_kJperMin = 750 
Qout_kJperSec = Qout_kJperMin/60

In [10]:
# define state 1

state1 = HumidAir().with_state(
    InputHumidAir.pressure(p1_Pa),
    InputHumidAir.temperature(T1_C),
    InputHumidAir.relative_humidity(phi_1_percent),
    #InputHumidAir.wet_bulb_temperature(Twb_C),
)

h1_JperKg = state1.enthalpy
h1_kJperKg = h1_JperKg/1e3; print(h1_kJperKg)
w1 = state1.humidity
v1_m3PerKg = state1.specific_volume; print(v1_m3PerKg)

75.10321439967704
0.8810477281821751


Let's do an energy balance for this steady flow device:

$$\dot{m} h_1 = \dot{Q}_\text{out} + \dot{m}h_2  $$

$$\implies h_2 = h_1 -\dot{Q}_\text{out}/\dot{m} $$

We need to know the mass flow rate.

In [19]:
Vdot_m3PerS = 3.1415*((D0_m/2)**2)*vel1_mPerS; print(Vdot_m3PerS)

1.2723075000000001


In [9]:
mdot_kgPerS = Vdot_m3PerS/v1_m3PerKg; print(mdot_kgPerS)

1.4440846497897375


In [11]:
h2_kJperKg = h1_kJperKg - (Qout_kJperSec/mdot_kgPerS); print(h2_kJperKg)

66.4472121342835


So now we have the enthalpy and pressure at state 2. However, we need one additional piece of information to be able to determine all other properties. 

Here let us assume that we are only doing "simple" cooling ($\phi_1 < \phi_2 < 100\%$). That is, there is no condensation occurring. If this is the case, then $$\omega_1 = \omega_2$$.

In [14]:
w2 = w1
p2_Pa = p1_Pa
h2_JperKg = h2_kJperKg*1000

In [22]:
# define state 2

state2 = HumidAir().with_state(
    InputHumidAir.pressure(p2_Pa),
    InputHumidAir.enthalpy(h2_JperKg),
    InputHumidAir.humidity(w2),
    #InputHumidAir.temperature(T1_C),
    #InputHumidAir.relative_humidity(phi_1_percent),
    #InputHumidAir.wet_bulb_temperature(Twb_C),
)

h2_JperKg = state2.enthalpy
h2_kJperKg = h2_JperKg/1e3; print(h2_kJperKg)
# w1 = state1.humidity
v2_m3PerKg = state2.specific_volume; print(v2_m3PerKg)

66.4472121342835
0.8567274884851074


In [23]:
print('(a) The exit temperature is, T2: {0:1.2f} C'.format(state2.temperature))

(a) The exit temperature is, T2: 26.52 C


In [24]:
print('(b) The exit relative humidity is: {0:1.2f} %'.format(state2.relative_humidity))

(b) The exit relative humidity is: 73.06 %


In [25]:
Vdot2_m3perS = mdot_kgPerS*v2_m3PerKg
area2_m2 = 3.1415*((D0_m/2)**2)
vel2_mPerS = Vdot2_m3perS/area2_m2

In [26]:
print('(c) The exit velocity is: {0:1.2f} m/s'.format(vel2_mPerS))

(c) The exit velocity is: 17.50 m/s
